In [1]:
import mlflow

mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("Customer Churn Prediction - Model Comparison")


C:\Users\Pratyush Basu\AppData\Roaming\Python\Python312\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/22 14:52:12 INFO mlflow.tracking.fluent: Experiment with name 'Customer Churn Prediction - Model Comparison' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:d:/Deloitte NLA project .. '
 'Customer/customer_churn_analytics/notebook/../mlruns/265673253714853429'), creation_time=1769073732038, experiment_id='265673253714853429', last_update_time=1769073732038, lifecycle_stage='active', name='Customer Churn Prediction - Model Comparison', tags={}>

In [2]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    recall_score,
    f1_score
)

from xgboost import XGBClassifier


import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv('../data/telco_customer_churn_clean_after_preprocessing.csv')
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,AvgChargesPerMonth,TotalServices,LongTermContract
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,14.925000,1,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.50,No,53.985714,2,1
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,36.050000,2,0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,40.016304,3,1
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,50.550000,0,0
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,91.166667,3,0
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,No,84.756522,2,0
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,Month-to-month,No,Mailed check,29.75,301.90,No,27.445455,1,0
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,105.036207,4,0
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,One year,No,Bank transfer (automatic),56.15,3487.95,No,55.364286,2,1


In [4]:
X = df.drop(columns=["customerID", "Churn"])
y = df["Churn"].map({"No": 0, "Yes": 1})


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [6]:
preprocessor = joblib.load("../models/preprocessor.pkl")


In [7]:
lr_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(
        class_weight="balanced",
        solver="liblinear",
        max_iter=5000,
        random_state=42
    ))
])
lr_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=5000,
                                    random_state=42, solver='liblinear'))])

In [8]:
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(
        class_weight="balanced",
        n_estimators=200,
        max_depth=10,
        random_state=42
    ))
])
rf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        n_estimators=200, random_state=42))])

In [9]:
# Calculate class imbalance ratio
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

In [10]:
xgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        eval_metric="logloss",
        random_state=42
    ))
])
xgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [11]:
THRESHOLD = 0.40  # We picked this because it gave 87% Recall

# Logistic Regression
lr_pred = lr_pipeline.predict(X_test)
lr_prob = lr_pipeline.predict_proba(X_test)[:, 1]

lr_pred_custom = (lr_prob >= THRESHOLD).astype(int)

# --- Verification ---
from sklearn.metrics import classification_report
print("Logistic Regression:",classification_report(y_test, lr_pred_custom))

# Random Forest
rf_pred = rf_pipeline.predict(X_test)
rf_prob = rf_pipeline.predict_proba(X_test)[:, 1]

rf_pred_custom = (rf_prob >= THRESHOLD).astype(int)

# --- Verification ---
from sklearn.metrics import classification_report
print("Random Forest:",classification_report(y_test, rf_pred_custom))

# XGBoost
xgb_pred = xgb_pipeline.predict(X_test)
xgb_prob = xgb_pipeline.predict_proba(X_test)[:, 1]

xgb_custom = (xgb_prob >= THRESHOLD).astype(int)

# --- Verification ---
from sklearn.metrics import classification_report
print("XGBoost:",classification_report(y_test, xgb_custom))


Logistic Regression:               precision    recall  f1-score   support

           0       0.93      0.64      0.76      1035
           1       0.47      0.87      0.61       374

    accuracy                           0.70      1409
   macro avg       0.70      0.75      0.68      1409
weighted avg       0.81      0.70      0.72      1409

Random Forest:               precision    recall  f1-score   support

           0       0.91      0.73      0.81      1035
           1       0.51      0.80      0.63       374

    accuracy                           0.75      1409
   macro avg       0.71      0.76      0.72      1409
weighted avg       0.81      0.75      0.76      1409

XGBoost:               precision    recall  f1-score   support

           0       0.91      0.70      0.79      1035
           1       0.49      0.80      0.61       374

    accuracy                           0.72      1409
   macro avg       0.70      0.75      0.70      1409
weighted avg       0.80      

In [12]:
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score

THRESHOLD = 0.40

results = {
    "Logistic Regression": {
        "train_accuracy": lr_pipeline.score(X_train, y_train),  
        "accuracy": accuracy_score(y_test, lr_pred_custom),
        "roc_auc": roc_auc_score(y_test, lr_prob),
        "recall": recall_score(y_test, lr_pred_custom),
        "f1": f1_score(y_test, lr_pred_custom)
    },
    "Random Forest": {
        "train_accuracy": rf_pipeline.score(X_train, y_train),
        "accuracy": accuracy_score(y_test, rf_pred_custom),
        "roc_auc": roc_auc_score(y_test, rf_prob),
        "recall": recall_score(y_test, rf_pred_custom),
        "f1": f1_score(y_test, rf_pred_custom)
    },
    "XGBoost": {
        "train_accuracy": xgb_pipeline.score(X_train, y_train),
        "accuracy": accuracy_score(y_test, xgb_custom),
        "roc_auc": roc_auc_score(y_test, xgb_prob),
        "recall": recall_score(y_test, xgb_custom),
        "f1": f1_score(y_test, xgb_custom)
    }
}

results


{'Logistic Regression': {'train_accuracy': 0.7525736599219027,
  'accuracy': 0.7012065294535131,
  'roc_auc': 0.8416518122400475,
  'recall': 0.8663101604278075,
  'f1': 0.6061739943872778},
 'Random Forest': {'train_accuracy': 0.8651047213347532,
  'accuracy': 0.7466288147622427,
  'roc_auc': 0.8379007465963988,
  'recall': 0.8021390374331551,
  'f1': 0.6269592476489029},
 'XGBoost': {'train_accuracy': 0.8496627618033369,
  'accuracy': 0.7246273953158269,
  'roc_auc': 0.8344868118525406,
  'recall': 0.8021390374331551,
  'f1': 0.6072874493927125}}

In [13]:
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

mlflow.set_experiment("Customer Churn Prediction - Model Comparison")

THRESHOLD = 0.40

models = {
    "Logistic Regression": (lr_pipeline, lr_prob, lr_pred_custom),
    "Random Forest": (rf_pipeline, rf_prob, rf_pred_custom),
    "XGBoost": (xgb_pipeline, xgb_prob, xgb_custom)
}

for model_name, (model, prob, pred_custom) in models.items():
    with mlflow.start_run(run_name=model_name):

        mlflow.log_param("model", model_name)
        mlflow.log_param("threshold", THRESHOLD)

        mlflow.log_metric("accuracy", accuracy_score(y_test, pred_custom))
        mlflow.log_metric("recall_churn", recall_score(y_test, pred_custom))
        mlflow.log_metric("f1_churn", f1_score(y_test, pred_custom))
        mlflow.log_metric("roc_auc", roc_auc_score(y_test, prob))

        # Log model artifact (optional for comparison)
        mlflow.sklearn.log_model(model, "model")

print("Model comparison logged in MLflow")


2026/01/22 14:52:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 14:52:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 14:52:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Model comparison logged in MLflow


In [14]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os

os.makedirs("cm_plots", exist_ok=True)

for model_name, (_, _, pred_custom) in models.items():

    cm = confusion_matrix(y_test, pred_custom)

    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")

    file_path = f"cm_plots/{model_name.replace(' ', '_')}_cm.png"
    plt.savefig(file_path)
    plt.close()

    with mlflow.start_run(run_name=f"{model_name}_Confusion_Matrix", nested=True):
        mlflow.log_artifact(file_path)

print("Confusion matrices logged as MLflow artifacts")


Confusion matrices logged as MLflow artifacts


In [15]:
mlflow.set_experiment("Customer Churn Prediction - Final")

with mlflow.start_run(run_name="Final_Logistic_Regression_Model"):

    mlflow.log_param("model", "Logistic Regression")
    mlflow.log_param("threshold", 0.40)
    mlflow.log_param("selection_reason", "Highest churn recall")

    mlflow.log_metric("accuracy", results["Logistic Regression"]["accuracy"])
    mlflow.log_metric("recall_churn", results["Logistic Regression"]["recall"])
    mlflow.log_metric("f1_churn", results["Logistic Regression"]["f1"])
    mlflow.log_metric("roc_auc", results["Logistic Regression"]["roc_auc"])

    mlflow.sklearn.log_model(lr_pipeline, "model")

print("Final model logged for deployment")


2026/01/22 14:52:58 INFO mlflow.tracking.fluent: Experiment with name 'Customer Churn Prediction - Final' does not exist. Creating a new experiment.
2026/01/22 14:52:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Final model logged for deployment


In [16]:
import mlflow
print(mlflow.get_tracking_uri())


file:../mlruns


In [17]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
for exp in client.search_experiments():
    print(exp.experiment_id, exp.name)


669676909816585381 Customer Churn Prediction - Final
265673253714853429 Customer Churn Prediction - Model Comparison
0 Default


#In CMD



#python -m mlflow ui

In [18]:
import joblib

FINAL_THRESHOLD = 0.40

joblib.dump(
    {
        "model": lr_pipeline,
        "threshold": FINAL_THRESHOLD
    },
    "../models/final_churn_model.pkl"
)


['../models/final_churn_model.pkl']

In [20]:
## Final Model Selection

#- Logistic Regression was selected as the final churn model due to its high recall for churn customers.
#- A probability threshold of **0.40** was chosen to prioritize identifying at-risk customers.
#- Although this increases false positives(roughly 1%), it reduces missed churners, aligning with retention-focused business objectives.
